# Agentic RAG

### pre-requisites

In [ ]:
%pip install yfinance langchain langchain_community langchain_ollama

### Tools

In [22]:
import yfinance as yf

# tool to get the stock price of a stock quote
def stock_market_tool(stock_quote: str):
    # get the ticker
    ticker = yf.Ticker(stock_quote)

    # get the current stock_price
    price = ticker.info['regularMarketPrice']

    return {"price": price}    


# create the metadata for the stock market tool
stock_market_tool_metadata = {
    "name": "company_stock_price",
    "description": "this tool is used to get the current stock price from stock market",
    "arguments": [{"stock_quote": "stock ticker symbol for a company"}],
    "response": "latest stock price of company"
}

In [23]:
# tool to get the current temperature/weather from a required city
def weather_tool(city: str):
    if city.lower() == "pune":
        return {"temperature": 36}
    elif city.lower() == "mumbai":
        return {"temperature": 38}
    elif city.lower() == "satara":
        return {"temperature": 34}
    else:
        return "not available"

# create the metadata for the weather tool
weather_tool_metadata = {
    "name": "city_weather",
    "description": "this tool is used to get the current temperature of required city",
    "arguments": [{"city": "name of the city"}],
    "response": "current temperature"
}

In [24]:
# creating the list of tools available
tools = [stock_market_tool_metadata, weather_tool_metadata]

# create the mapping between the tool name and its function
tools_mapping = {
    'city_weather': weather_tool,
    'company_stock_price': stock_market_tool
}

### LLM configration

In [25]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

template = """
You are a helpful assistant capable of answering questions on various topics. 
You must not use your internal knowledge or information to answer questions.

Instructions:

Think step-by-step to create a plan.
Use only the following available tools to find information.
Tools Available:

{tools}
Guidelines for Responses:

Format 1: If the question cannot be answered with the available tools, use this format:
{{"answer": "No appropriate tool available"}}

Format 2: If you need to run tools to obtain the information, use this format:
{{"actions": [{{ "action" : tool name, "arguments" : dictionary of argument values}}}}]

Format 3: If you can answer the question using the responses from the tools, use this format:
{{"answer": "your response to the question", "explanation": "provide your explanation here"}}


Avoid any preamble; respond directly using one of the specified JSON formats.
Question:

{question}
Tool Responses:

{tool_responses}
Your Response:
"""


# create the prompt template
prompt_template = ChatPromptTemplate.from_template(template=template)

# create the llm model
model = ChatOllama(model="llama3")

In [41]:
def invoke_tool(response):
    # get the result of every action
    action_responses = []

    # check if the response has at least an action present
    if len(response['actions']) == 0:
        print({"answer": "question can not answered as there is no tool to answer the question"})
    else:
        
        # perform all the actions
        for action in response['actions']:
            
            # get the function to invoke
            function = tools_mapping[action['action']]

            # pass the arguments and call the function
            if type(action['arguments']) == list:
                result = function(**action['arguments'][0])
                action['response'] = result
            elif type(action['arguments']) == dict:
                result = function(**action['arguments'])
                action['response'] = result

            # collect the action with the result
            action_responses.append(action)
    
    return action_responses 

### step 1: get tool to be used

In [57]:
import json

question = "I have only $100, can I purchase 2 stocks of GOOG?"

# create the prompt
prompt = prompt_template.invoke({
    "question": question, 
    "tools": tools,
    "tool_responses": ""
})

# get the answer from LLM
response = model.invoke(prompt)

# convert the answer to the JSON
response_json = json.loads(response.content)
print(f"STEP 1: \n", response_json, "\n")

STEP 1: 
 {'actions': [{'action': 'company_stock_price', 'arguments': {'stock_quote': 'GOOG'}}]} 



### step 2: invoke the tool and get the result           

In [60]:
# check if we already have gottent the answer
if 'answer' in response_json:
    print(f"got an answer: {response_json['answer']}")
elif 'actions' in response_json:
    tool_responses = invoke_tool(response_json)
    print(f"STEP 2: \n", responses, "\n")

STEP 2: 
 [{'action': 'company_stock_price', 'arguments': {'stock_quote': 'AAPL'}, 'response': {'price': 219.9202}}] 



### step 3: polish the result and present the answer to the user

In [63]:
# create the prompt
prompt = prompt_template.invoke({
    "question": question, 
    "tools": tools,
    "tool_responses": response_json
})

# get the answer from LLM
response = model.invoke(prompt)

print(response.content)

{"actions": [
   {"action": "company_stock_price", 
    "arguments": {"stock_quote": "GOOG"}
   },
   {"action": "company_stock_price", 
    "arguments": {"stock_quote": "GOOG"}
   }
]}

Note: The available tool is 'company_stock_price' which returns the current stock price. To answer this question, I need to use this tool twice to get the current stock price of GOOG and then calculate if it's possible to purchase 2 stocks with $100.
